In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [109]:
train_df=pd.read_csv("datas/train.csv",encoding="gb2312")
test_df=pd.read_csv("datas/testA.csv",encoding="gb2312")

In [110]:
def encoded(train_df):
    sex_encoded, sex_categories = train_df["性别"].factorize()
    train_df["性别"]=sex_encoded
    date_encoded, date_categories = train_df["体检日期"].factorize()
    train_df["体检日期"]=date_encoded
    return train_df

In [111]:
encoded(train_df)
median=train_df.mean()
tem_df=train_df.fillna(median)
corr_matrix=tem_df.corr()
relative_df=corr_matrix["血糖"]
impotantcolumns=relative_df[relative_df.values>0.2].index.tolist()

In [112]:
relative_df.sort_values(ascending=False)

血糖             1.000000
年龄             0.252862
甘油三酯           0.227922
血红蛋白           0.148011
低密度脂蛋白胆固醇      0.146213
红细胞平均血红蛋白浓度    0.146058
总胆固醇           0.144753
*碱性磷酸酶         0.136771
尿素             0.136593
红细胞计数          0.121187
红细胞压积          0.120455
*丙氨酸氨基转换酶      0.117554
*r-谷氨酰基转换酶     0.112472
*天门冬氨酸氨基转换酶    0.089652
肌酐             0.087528
白细胞计数          0.087135
红细胞平均血红蛋白量     0.067214
中性粒细胞%         0.046148
id             0.045757
*球蛋白           0.043474
血小板体积分布宽度      0.040734
*总蛋白           0.040091
嗜碱细胞%          0.024687
血小板平均体积        0.022887
尿酸             0.019934
乙肝表面抗原         0.013570
单核细胞%          0.004719
嗜酸细胞%          0.002343
白蛋白            0.002285
乙肝e抗原          0.000469
乙肝核心抗体         0.000460
红细胞平均体积       -0.006325
体检日期          -0.008980
白球比例          -0.012292
乙肝e抗体         -0.013443
乙肝表面抗体        -0.025626
淋巴细胞%         -0.052161
血小板比积         -0.064953
血小板计数         -0.072670
高密度脂蛋白胆固醇     -0.073361
红细胞体积分布宽度     -0.075451
性别            -0

In [113]:
impotantcolumns.remove("血糖")

In [114]:
impotantcolumns

['年龄', '甘油三酯']

In [115]:
new_train_df=train_df[impotantcolumns]
new_test_df=test_df[impotantcolumns]
df=pd.concat([new_train_df,new_test_df])

In [116]:
df.fillna(df.median(),inplace=True)

In [117]:
df

,年龄,甘油三酯
0,41,1.31
1,41,2.81
2,46,0.99
3,22,1.06
4,48,0.97
5,74,2.45
6,31,1.51
7,55,1.32
8,39,2.07
9,35,1.25


In [42]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

lgb_train = lgb.Dataset(X_train, y_train)


NameError: name 'X_train' is not defined

In [ ]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

In [ ]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [181]:
train_feat=df[0:5642]


test_feat=df[5642:]

In [184]:
train_feat["血糖"]=pd.read_csv("datas/train.csv",encoding="gb2312")["血糖"]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [168]:
predictors=train_df.columns.tolist()

In [170]:
predictors.remove("血糖")

In [172]:
params = {
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mse',
    'sub_feature': 0.7,
    'num_leaves': 60,
    'colsample_bytree': 0.7,
    'feature_fraction': 0.7,
    'min_data': 100,
    'min_hessian': 1,
    'verbose': -1,
}

In [174]:
def evalerror(pred, df):
    label = df.get_label().values.copy()
    score = mean_squared_error(label,pred)*0.5
    return ('mse',score,False)

In [187]:
print('开始CV 5折训练...')
scores = []

train_preds = np.zeros(train_feat.shape[0])
test_preds = np.zeros((test_feat.shape[0], 5))
kf = KFold(len(train_feat), n_folds = 5, shuffle=True, random_state=520)
for i, (train_index, test_index) in enumerate(kf):
    print('第{}次训练...'.format(i))
    train_feat1 = train_feat.iloc[train_index]
    train_feat2 = train_feat.iloc[test_index]
    lgb_train1 = lgb.Dataset(train_feat1[predictors], train_feat1['血糖'])
    lgb_train2 = lgb.Dataset(train_feat2[predictors], train_feat2['血糖'])
    gbm = lgb.train(params,
                    lgb_train1,
                    num_boost_round=3000,
                    valid_sets=lgb_train2,
                    verbose_eval=100,
                    feval=evalerror,
                    early_stopping_rounds=100)
    feat_imp = pd.Series(gbm.feature_importance(), index=predictors).sort_values(ascending=False)
    train_preds[test_index] += gbm.predict(train_feat2[predictors])
    test_preds[:,i] = gbm.predict(test_feat[predictors])
print('线下得分：    {}'.format(mean_squared_error(train_feat['血糖'],train_preds)*0.5))


开始CV 5折训练...
第0次训练...
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 3.09321	valid_0's mse: 1.5466
[200]	valid_0's l2: 3.00322	valid_0's mse: 1.50161
[300]	valid_0's l2: 2.96706	valid_0's mse: 1.48353
[400]	valid_0's l2: 2.95984	valid_0's mse: 1.47992
[500]	valid_0's l2: 2.96248	valid_0's mse: 1.48124
Early stopping, best iteration is:
[408]	valid_0's l2: 2.95962	valid_0's mse: 1.47981
第1次训练...
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 2.07586	valid_0's mse: 1.03793
[200]	valid_0's l2: 2.03183	valid_0's mse: 1.01591
[300]	valid_0's l2: 2.01487	valid_0's mse: 1.00743
[400]	valid_0's l2: 2.01265	valid_0's mse: 1.00632
Early stopping, best iteration is:
[397]	valid_0's l2: 2.01257	valid_0's mse: 1.00628
第2次训练...
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 1.65319	valid_0's mse: 0.826593
[200]	valid_0's l2: 1.61016	valid_0's mse: 0.805078
[300]	valid_0's l2: 1.60457	valid_0's

In [118]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [119]:
X=scaler.transform(df)

In [120]:
X.shape

(6642, 2)

In [121]:
df

,年龄,甘油三酯
0,41,1.31
1,41,2.81
2,46,0.99
3,22,1.06
4,48,0.97
5,74,2.45
6,31,1.51
7,55,1.32
8,39,2.07
9,35,1.25


In [43]:
from sklearn import preprocessing
X = df.values
X = preprocessing.scale(X)

In [44]:
X.shape

(6642, 8)

In [149]:
x_train = X[0:5642]
x_test = X[5642:]

In [123]:
x_train[0]

array([ 0.42222222,  0.0251816 ])

In [87]:
x_train[0]

array([ 0.42222222,  0.21805089,  0.0251816 ,  0.13863514,  0.23913043,
        0.36753574,  0.69246575,  0.425     ])

In [124]:
y_train=train_df["血糖"].values

In [89]:
y_train

array([ 6.06,  5.39,  5.59, ...,  5.24,  6.37,  6.  ])

In [125]:
from sklearn.preprocessing import PolynomialFeatures

poly_features=PolynomialFeatures(degree=2,include_bias=False)

X_poly=poly_features.fit_transform(x_train)

In [127]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [128]:
lin_reg=LinearRegression()
lin_reg.fit(X_poly,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [129]:
mse=cross_val_score(lin_reg,X_poly,y_train,scoring="neg_mean_squared_error",cv=10)

In [130]:
np.mean(mse)

-2.1205079245853695

In [148]:
x_test[0]

array([ 0.56666667,  0.01937046])

In [146]:
X_test_poly = poly_features.fit_transform(x_test)

In [147]:
X_test_poly[0]

array([  5.66666667e-01,   1.93704600e-02,   3.21111111e-01,
         1.09765940e-02,   3.75214722e-04])

In [150]:
np.mean(lin_reg.predict(X_test_poly))

5.6770342242414884

In [140]:
result=pd.DataFrame({'pred':lin_reg.predict(X_test_poly).tolist()})

In [141]:
result.to_csv('submit.csv',index=False,header=None)